In [ ]:
import numpy as np
import pandas as pd
import glob
import sqlite3
import re
import os
import resource

In [ ]:
print("Opening a SQLite database at `lineage_fitness.sqlite` ")
db_lf = sqlite3.connect("lineage_fitness.sqlite")

In [ ]:
def read_lineage_fitness(csv_path):
    base_name = re.sub("_PPI_barcodes_fitness_counts.csv","",os.path.basename(csv_path))
    print("Beginning to read and process lineage fitness of ",base_name)
    this_datar = pd.read_csv(csv_path)
    print("Read CSV for ",base_name)
    this_datar.rename(columns={
            "Fitness_estimaion_error(d)":"Error",
            "Fitness_estimaion_error":"Error"
        },inplace=True)
    this_datar['index'] = this_datar.index
    this_datar.drop(
        ["Number_of_Barcodes","Barcode_sequences"]+
            list(this_datar.filter(regex="Counts_")),
        axis=1,inplace=True)
    this_datar["Experiment"] = base_name
    print(this_datar)
    return this_datar
#,nrows=100)\
#for debugging, this makes it run quite a bit faster...

In [ ]:
for this_csv in glob.glob('*_PPI_*.csv'):
    with db_lf:
        print('Memory usage: %s (kb)' % resource.getrusage(resource.RUSAGE_SELF).ru_maxrss)
        read_lineage_fitness(this_csv)\
            .to_sql("lineage_fitness",db_lf,if_exists="append",chunksize=10000,index=False)

print("Database on disk reports these heads for counts:")
print(pd.read_sql_query("SELECT * FROM lineage_fitness LIMIT 5",db_lf))